In [43]:
from cdx.cdx_helper import cdx_scan, CRAWL_CDX

total_bytes = 0
i = 0
j = 0

# Keep track of the current URL line:
url_ok = None
# If URLs appear to disappear, add them here:
#urls_ded = []

# Okay, needs to be a bit cleverer and not pick up on transient 404s (or possibly class them differently).
# i.e. if you hit a 200, that resets the 404. The first 404 sets up the pair, 
# but the pair is only reported if we hit the end of the set of results for that URL.

bbc = 'https://www.bbc.co.uk/news/'
mirror = 'https://www.mirror.co.uk/'
express = 'https://www.express.co.uk/'
guardian = 'https://www.theguardian.com/'

#prefixes = [bbc, mirror, express, guardian]
prefixes = [guardian]

for prefix in prefixes:
    for cdx in cdx_scan(prefix, cdx_service=CRAWL_CDX, limit=100_000_000):
        if cdx.statuscode == '200':
            total_bytes += int(cdx.length)
            if 'html' in cdx.mimetype:
                url_ok = cdx
                i += 1
        #elif int(int(cdx.statuscode)/100) == 4: # mostly interested in 404 but considering 4xx - too many 403?!?
        elif cdx.statuscode in ['404', '410', '451']:
            if 'html' in cdx.mimetype and url_ok and cdx.original == url_ok.original:

                urls_ded.append( (cdx, url_ok) )
                # Drop the OK so we don't add the same URL many times:
                url_ok = None
                #if cdx.urlkey in urls_ok:
                #    urls_ded[cdx.urlkey] = (cdx, urls_ok.pop(cdx.urlkey))
                j += 1
        #if i%100000 == 0:
        #    print(i, total_bytes, j)
        #print(cdx.length, cdx.original, cdx.statuscode)

#for pair in urls_ded:
#    ded, ok = pair
#    print(ded.original, ded.statuscode, ded.timestamp, ok.statuscode, ok.timestamp)


In [44]:
import pandas as pd
from urllib.parse import urlparse

d = []

for pair in urls_ded:
    ded, ok = pair
    d.append( {
      'url': ok.original,
      'host': urlparse(ok.original).hostname,
      'last_seen_ts': ok.timestamp,
      'last_seen_date': pd.to_datetime(ok.timestamp, format='%Y%m%d%H%M%S'),
      'gone_ts': ded.timestamp,
      'gone_date': pd.to_datetime(ded.timestamp, format='%Y%m%d%H%M%S'),
      'gone_status_code': int(ded.statuscode),
      'wb_url': 'https://dev.webarchive.org.uk/act/wayback/archive/%s/%s' % (ok.timestamp, ok.original),
    } )

df = pd.DataFrame(d)

df

,url,host,last_seen_ts,last_seen_date,gone_ts,gone_date,gone_status_code,wb_url
0,https://www.bbc.co.uk/news/24758587,www.bbc.co.uk,20190404133849,2019-04-04 13:38:49,20201104203118,2020-11-04 20:31:18,404,https://dev.webarchive.org.uk/act/wayback/arch...
1,https://www.bbc.co.uk/news/ampstories/brexitti...,www.bbc.co.uk,20181125121632,2018-11-25 12:16:32,20181129114515,2018-11-29 11:45:15,404,https://dev.webarchive.org.uk/act/wayback/arch...
2,https://www.bbc.co.uk/news/ampstories/climates...,www.bbc.co.uk,20181215130553,2018-12-15 13:05:53,20190204015514,2019-02-04 01:55:14,404,https://dev.webarchive.org.uk/act/wayback/arch...
3,https://www.bbc.co.uk/news/ampstories/elchapo/...,www.bbc.co.uk,20190103024611,2019-01-03 02:46:11,20190105121153,2019-01-05 12:11:53,404,https://dev.webarchive.org.uk/act/wayback/arch...
4,https://www.bbc.co.uk/news/ampstories/marsland...,www.bbc.co.uk,20181213152012,2018-12-13 15:20:12,20190102155705,2019-01-02 15:57:05,404,https://dev.webarchive.org.uk/act/wayback/arch...
...,...,...,...,...,...,...,...,...
610,https://www.theguardian.com/uk-news/video/2019...,www.theguardian.com,20190525120151,2019-05-25 12:01:51,20190526131859,2019-05-26 13:18:59,410,https://dev.webarchive.org.uk/act/wayback/arch...
611,https://www.theguardian.com/unwindireland/acro...,www.theguardian.com,20180906202428,2018-09-06 20:24:28,20190128183900,2019-01-28 18:39:00,410,https://dev.webarchive.org.uk/act/wayback/arch...
612,https://www.theguardian.com/visit-swedish-lapl...,www.theguardian.com,20180906202325,2018-09-06 20:23:25,20190128182930,2019-01-28 18:29:30,410,https://dev.webarchive.org.uk/act/wayback/arch...
613,https://www.theguardian.com/world/russia/2019/...,www.theguardian.com,20190609085128,2019-06-09 08:51:28,20190610083623,2019-06-10 08:36:23,404,https://dev.webarchive.org.uk/act/wayback/arch...


In [53]:
df.to_csv('thedead.csv', index=False)

In [52]:
import altair as alt

alt.Chart(df).mark_tick(thickness=4).encode(
    x = 'gone_date:T',
    y = 'host:O',
    color = 'gone_status_code:N',
    href = 'wb_url:N',
    tooltip=['url', 'last_seen_date', 'gone_date', 'gone_status_code']
).configure_mark(
    opacity=0.4
).properties(
    width=600
).interactive()

alt.Chart(...)